In [1]:
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import pytesseract
import numpy as np
from utils import needed_values
import pandas as pd
from rules_english import results
from pdf2image import convert_from_path

pytesseract.pytesseract.tesseract_cmd  = "/usr/bin/tesseract"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
def detect_text(path):
    if path[-3:].lower() == "pdf":
        pages = convert_from_path(path, 500)
        pages[0].save(path+'.jpg', 'JPEG')
        path = path+'.jpg'


    image = Image.open(path)


    if path[-3:].lower() == "tif":
        image = image.convert("RGB")



    data = pytesseract.image_to_data(image)
    with open("data.txt", "wb") as text_file: 
        text_file.write(str(data).encode('utf-8'))               

    df = pd.read_csv("data.txt",  error_bad_lines=False, sep='\t')



    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("DejaVuSans", 25)
    
    data = []

    df.replace(' ', np.nan, inplace=True)
    df.dropna(axis=0, how='any', inplace=True)
    df.reset_index(drop = True, inplace = True)


    for idx, row in df.iterrows():
        draw.rectangle((row['left'], row['top'], 
                            row['left'] + row['width'], 
                            row['top'] + row['height']), 
                        outline='green')
        #draw.text((row['left'], row['top']+12), str(row['text']) + " Confidence:" + str(row['conf']), fill='red', font=font)

        data.append({"Text": str(row['text']), "conf": row['conf'],
                         "height": row['height'], "line": 0, 
                         "x0": row['left'] , "y0": row['top'], 
                         "x2" :(row['left'] + row['width']), 
                         "y2": (row['top'] + row['height'])})

        

    df = pd.DataFrame(data)


    df.sort_values('y0', inplace = True)
    df = df.loc[df.height < 50]
    df = df.reset_index(drop=True)
    prev_y = 0
    prev_height = -1
    line = 0


    for index, row in df.iterrows():
        if row.y0 - prev_y > prev_height:
            line += 1
            df.iloc[index, df.columns.get_loc('line')] = line
        else:
            df.iloc[index, df.columns.get_loc('line')] = line

        prev_y = row.y0
        prev_height = row.height


    text = df.copy()


    output, table, green_indices = results(text)
    


    for idx in green_indices:
        row = text.iloc[idx]

        draw.rectangle((row['x0'], row['y0'], row['x2'], row['y2']), outline='blue')


        draw.text((row['x0'], row['y0']-20), str(row.Text) + " :" + str(row.conf)+"%", fill='red', font=font)
        
    

    data_needed = []
    for key, value in output.items():
        data_needed.append({'Text': key, 'Value': value})


    df_needed = pd.DataFrame(data_needed)
    
    path += '-remittance.jpg'

    image.save(path)


        

    return df_needed, table, path[7:]


In [32]:
detect_text('tmp/2.tif')

/notebooks/rules_english.py:150: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  amount.extend(text.loc[text.line == line].loc[text['Text'].str.contains("\d+(\.\d{0,1,2}){0,1}$")].Text.values)
/notebooks/rules_english.py:104: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  value = line_df.loc[text['Text'].str.contains("\d+(\.\d{1,2})$")].Text.values[0]


(           Text       Value
 0     Sort Code   55-61-01.
 1   Payment No.  U113423668
 2          Date  2017-04-21
 3   Account No.    05200105
 4  Total Amount     77083.3,          Date Other   Reference     Value
 0  2017-04-21        U113423668       NaN
 1  2017-04-21        U113423669    196.74
 2  2017-04-21        U113423670  41159.27
 3  2017-04-21        U113423671   1988.53, 'if-remittance.jpg')